In [6]:
from sim import Point2D, Environment2D

In [7]:
# Source setup

source_point = Point2D(0., 0.)
source_ = 0.

In [8]:
id(source_point)

140172583365168

In [11]:
# List reference proof

class Foo(object):
    def __init__(self,val):
        self.value=val
    def changeval(self,newval):
        self.value=newval

a = Foo(2)
b = [a]
print(b[0].value)

a.changeval(3)
print(b[0].value)

2
3
